In [14]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform
from scipy.stats import norm
from scipy.stats import binom
import seaborn as sns
import pandas as pd
from sklearn.utils import shuffle
import os

In [15]:
print(os.getcwd())
cwd=os.getcwd()

/Users/chenzhenxu/jupyter_notebook


In [16]:
#https://www.kaggle.com/c/titanic/data

In [17]:
train_data = pd.read_csv(cwd+"/titanic/train.csv")
test_data=pd.read_csv(cwd+"/titanic/test.csv")
print(train_data.shape)
train_data.head()


(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [19]:
y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features]).to_numpy()
#X_test = pd.get_dummies(test_data[features])

In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=44)

In [21]:
X_train.shape

(712, 5)

In [22]:
logreg = LogisticRegression()
logreg = logreg.fit(X_train,y_train)
# Predicting the values
y_pred=logreg.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)


[[88 18]
 [29 44]]


0.7374301675977654

In [23]:
class lr:
    def __init__(self, lr=1.0, itrs=100):
        self.lr=lr
        self.itrs=itrs
    def h(self, datx):
        f=datx@self.w
        return 1/(1+np.exp(-f))
    def cost(self, X, Y):
        H=self.h(X)
        m=len(Y)
        cost = np.transpose(-Y)@np.log(H) - np.transpose(1-Y)@np.log(1-H) 
        cost = (1/m)*cost
        return cost
    def costgrad(self, X,Y):
        H=self.h(X)
        m=len(Y)
        dev = np.transpose(X)@(H - Y)
        dev = (1/m)*dev
        return dev
    def fit(self, X,Y):
        m,n=X.shape
        self.w=np.ones(n)
        costs=[]
        for itr in range(self.itrs):
            self.w=self.w-self.lr*self.costgrad(X,Y)
            cos=self.cost(X,Y)
            costs.append(cos)
            print(cos)
    def predict(self, xtest):
        p=self.h(xtest)
        return 1*(p>0.5)
    
        

In [26]:
## multinomial Logistic regression: 2nd order gradient+hessian implementation
class mlr_hess:
    def __init__(self, lr=0.1, itrs=100):
        self.lr=lr
        self.itrs=itrs
    def h(self, datx):
        ## return the probability for each class, each row per sample
        A=datx@self.w.transpose() 
        ## (N,D)x(DxC)=>shape of (NxC), 
        ## next apply softmax function 
        N,D=A.shape
        P=np.zeros_like(A)
        for i in range(N):
            lse=self.lse(A[i,:])
            P[i,:]=np.array([np.exp(a-lse) for a in A[i,:]])
        return P
            
    def cost(self, X, Y):
        P=self.h(X)
        totcost=0
        for i in range(self.N):
            yvec=Y[i,:]
            pvec=P[i,:]
            cost = self.ce(yvec,pvec)
            totcost+=cost
        return (1.0/self.N)*totcost
        
    def costgrad(self, X,Y):
        P=self.h(X)
        print(f"shape of P :{P.shape}")
        gw=np.zeros((self.D, self.C))
        for i in range(self.N):
            xi=np.reshape(X[i,:],(-1,1))
            dif=P[i,:]-Y[i,:]
            dif2=np.reshape(dif,(-1,1))
            res=np.matmul(xi,dif2.transpose())
            gw+=res
        gw= (1/self.N)*gw
        return [gw,P]
        
    def fit(self, X,Y):
        self.N,self.D=X.shape
        ones=np.ones((self.N,1))
        X2=np.hstack((X,ones))
        # we assume X did not prepend 1's to the first column 
        self.D=self.D+1 
        self.C=len(list(set(Y)))
        ## also need to transform Y to 2d array
        Y2=np.zeros((self.N,self.C))
        for i in range(self.N):
            Y2[i,Y[i]]=1
        print(f"input data: size={self.N}, dim={self.D}")
        print(f"input Y data: # of classes={self.C}")
        ## initialize weight matrix
        self.w=np.zeros((self.C,self.D))
        gw,P=self.costgrad(X2,Y2)
        print(f"gw shape:{gw.shape}")
        
        ## update
        costs=[]
        for itr in range(self.itrs):
            for c in range(self.C):
                #hessian
                H=np.zeros((self.D,self.D))
                for i in range(self.N):
                    xi=np.reshape(X2[i,:],(self.D,-1))
                    xx=np.matmul(xi,xi.transpose())
                    H+=P[i,c]*xx
                H=np.linalg.inv((1.0/self.N)*H)
                self.w[c,:]=self.w[c,:]-self.lr*np.matmul(H,gw[:,c])
            cost=self.cost(X2,Y2)
            if itr>1 and cost>costs[-1]:
                break
            costs.append(cost)
        print("costs:")
        print(costs)
            
    def predict(self, xtest):
        N,D=xtest.shape
        ones=np.ones((N,1))
        Xnew=np.hstack((xtest,ones))
        P=self.h(Xnew)
        ## for each row, let the max column be 1
        #preds = np.zeros_like(P)
        #preds[np.arange(N), preds.argmax(1)] = 1
        preds=P.argmax(1) 
        print(f"shape of preds:{preds.shape}")
        return preds
    
    def lse(self, V):
        ## V is a vector of dim C, apply log-sum-exp to V, return the scalar result
        res=np.log(np.exp(V).sum())
        return res
    
    def ce(self,u,v):
        ## cross entropy
        eps=0.001
        v+=eps
        res=-np.dot(u,np.log(v))
        return res
        

In [27]:
dataset = pd.read_csv("/Users/arthur/work/pn2022/PN/data/wine/wine-m/train.csv")
dataset.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/arthur/work/pn2022/PN/data/wine/wine-m/train.csv'

In [28]:
# Splitting datset to X and y axis 

X=dataset.iloc[: , 1:14].values
y=dataset.iloc[: ,-1].values

NameError: name 'dataset' is not defined

In [144]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=44)

In [191]:
#Feature Scaling

from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [192]:
X_train.shape, X_test.shape

((427, 13), (107, 13))

In [255]:
## base model logistic regression
logreg = LogisticRegression(penalty='none',multi_class='multinomial')

#logreg = LogisticRegression(penalty='elasticnet', solver='saga',l1_ratio=0.1)
logreg.fit(X_train,y_train)
# Predicting the values
y_pred=logreg.predict(X_test)
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
# Calculating quality for model

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)


[[35  1  0]
 [ 0 41  2]
 [ 0  0 28]]


0.9719626168224299

In [256]:
logreg.coef_

array([[ 10.20300696,   1.6930989 ,   6.943174  ,  -8.41799171,
          0.52146428,  -2.98980975,   6.28364942,  -4.5024047 ,
         -2.95697049,  -7.24990097,  -0.92044785,   6.10570544,
          9.46564576],
       [-10.87783018,  -2.17212502, -12.25744095,   7.30772657,
         -0.80397581,   1.28453893,   4.7682052 ,   5.71005626,
          1.42798769,  -3.96925927,   8.56102894,   1.08569621,
        -10.59292482],
       [  0.67482322,   0.47902612,   5.31426695,   1.11026514,
          0.28251153,   1.70527082, -11.05185462,  -1.20765157,
          1.5289828 ,  11.21916024,  -7.64058109,  -7.19140165,
          1.12727906]])

In [259]:
from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99        35
           1       0.95      0.98      0.96        42
           2       1.00      0.93      0.97        30

    accuracy                           0.97       107
   macro avg       0.98      0.97      0.97       107
weighted avg       0.97      0.97      0.97       107



In [272]:
## our implementation
logreg = mlr()

#logreg = LogisticRegression(penalty='elasticnet', solver='saga',l1_ratio=0.1)
logreg.fit(X_train,y_train)
# Predicting the values
y_pred=logreg.predict(X_test)
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
# Calculating quality for model

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)


input data: size=427, dim=14
input Y data: # of classes=3
shape of P :(427, 3)
gw shape:(14, 3)
costs:
[0.27978887896202537, 0.1908361949148315]
shape of preds:(107,)
[[34  2  0]
 [ 3 33  7]
 [ 0  0 28]]


0.8878504672897196

In [283]:
## our implementation
logreg = mlr_hess()

#logreg = LogisticRegression(penalty='elasticnet', solver='saga',l1_ratio=0.1)
logreg.fit(X_train,y_train)
# Predicting the values
y_pred=logreg.predict(X_test)
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
# Calculating quality for model

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)


input data: size=427, dim=14
input Y data: # of classes=3
shape of P :(427, 3)
gw shape:(14, 3)
costs:
[0.9353356912266297, 0.7928947077999113, 0.6685859992480612, 0.5618630581371131, 0.47148775072571464, 0.3957747112952101, 0.33283916105477934, 0.2807931705325173, 0.23787449890179604, 0.20251514372002657, 0.17336557552757287, 0.14929094204066604, 0.12935212005859253, 0.11278036882264995, 0.09895086642488764, 0.0873579461935702, 0.07759328509035349, 0.06932738996520921, 0.06229425204813161, 0.05627882440340935, 0.051106907342713774, 0.046637032654343866, 0.04275397802453297, 0.039363595692117906, 0.03638869262690465, 0.0337657480111342, 0.03144229556259057, 0.029374833022017887, 0.027527149494393294, 0.025868984147933222, 0.024374947954959664, 0.023023654558548778, 0.021797017710294478, 0.020679681668155907, 0.01965855797579867, 0.018722447573047724, 0.01786173153453668, 0.017068117155632414, 0.016334428801744946, 0.0156544350661902, 0.015022705465568408, 0.0144344912359833, 0.01388562

0.9719626168224299

In [69]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

In [287]:
from sklearn.datasets import load_boston
boston = load_boston()
print(boston.data.shape)

(506, 13)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets inclu

In [288]:
boston.data.shape

(506, 13)

In [65]:
type(boston.data)

numpy.ndarray

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(boston.data,boston.target,test_size=0.2, random_state=44)

In [73]:
lm.fit(X_train, y_train)

LinearRegression()

In [74]:
ypred=lm.predict(X_test)

In [76]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, ypred)
print("MSE: %.4f" % mse)


MSE: 0.1846


In [79]:
len(boston.target)

506